In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import prettytable
prettytable.DEFAULT = prettytable.PLAIN_COLUMNS

# Exploring the input directory to find the database
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load SQL magic extension
%load_ext sql

# Connect to the SQLite database
# Replace the path below with the actual path to your SQLite database file
%sql sqlite:///C:/Users/chojk/Desktop/myproject/KaggleTennisDataProject/database.sqlite

In [4]:
%%sql
-- All-time Top Players based on points
SELECT
    p.name_first || ' ' || p.name_last AS full_name, 
    MIN(r.rank) AS best_rank,
    MAX(r.points) AS max_points
FROM
    rankings r
JOIN
    players p
ON
    r.player = p.player_id
GROUP BY
    p.name_first, p.name_last
HAVING
    MAX(r.points) IS NOT NULL
ORDER BY
    max_points DESC,
    best_rank ASC
LIMIT 10;



 * sqlite:///C:/Users/chojk/Desktop/myproject/KaggleTennisDataProject/database.sqlite
Done.


full_name,best_rank,max_points
Novak Djokovic,1,16950.0
Rafael Nadal,1,15390.0
Andy Murray,1,12685.0
Roger Federer,1,12315.0
Daniil Medvedev,1,10780.0
Carlos Alcaraz,1,9815.0
Dominic Thiem,3,9125.0
Jannik Sinner,2,8860.0
Stefanos Tsitsipas,3,8350.0
Alexander Zverev,2,8240.0


In [6]:
%%sql
-- Point-Based Yearly Player Ranking
WITH RankedPlayers AS (
    SELECT
        SUBSTR(CAST(r.ranking_date AS STRING), 1, 4) AS ranking_year,
        r.player,
        p.name_first || ' ' || p.name_last AS full_name,
        r.rank,
        MAX(r.points) AS points,  -- Use MAX to ensure no duplicates
        ROW_NUMBER() OVER (
            PARTITION BY SUBSTR(CAST(r.ranking_date AS STRING), 1, 4) 
            ORDER BY r.rank ASC
        ) AS player_rank
    FROM
        rankings r
    JOIN
        players p
    ON 
        r.player = p.player_id
    GROUP BY
        ranking_year, r.player, p.name_first, p.name_last, r.rank
)
SELECT DISTINCT
    ROW_NUMBER() OVER (
        PARTITION BY ranking_year ORDER BY points DESC
    ) AS rank_in_the_year,  
    ranking_year,
    player,
    full_name,
    points
FROM
    RankedPlayers
WHERE
    player_rank <= 3
ORDER BY
    ranking_year DESC,
    rank_in_the_year;


 * sqlite:///C:/Users/chojk/Desktop/myproject/KaggleTennisDataProject/database.sqlite
Done.


rank_in_the_year,ranking_year,player,full_name,points
1,2024,104925,Novak Djokovic,11245.0
2,2024,207989,Carlos Alcaraz,9255.0
3,2024,206173,Jannik Sinner,8860.0
1,2023,104925,Novak Djokovic,11795.0
2,2023,207989,Carlos Alcaraz,9815.0
3,2023,104745,Rafael Nadal,6020.0
1,2022,104925,Novak Djokovic,11540.0
2,2022,106421,Daniil Medvedev,8615.0
3,2022,207989,Carlos Alcaraz,6820.0
1,2021,104925,Novak Djokovic,12133.0


In [5]:
%%sql
-- Calculate all-time winning percentage for Right-Handed vs Left-Handed players
SELECT
    'Winning Statistics' AS metric,
    SUM(CASE WHEN winner_hand = 'L' AND loser_hand = 'R' THEN 1 ELSE 0 END) AS "Left-Handed Wins",
    ROUND(100.0 * SUM(CASE WHEN winner_hand = 'L' AND loser_hand = 'R' THEN 1 ELSE 0 END) /
          SUM(CASE WHEN (winner_hand = 'L' AND loser_hand = 'R') OR (winner_hand = 'R' AND loser_hand = 'L') THEN 1 ELSE 0 END), 2) AS "Left-Handed Winning %",
    SUM(CASE WHEN winner_hand = 'R' AND loser_hand = 'L' THEN 1 ELSE 0 END) AS "Right-Handed Wins",
    ROUND(100.0 * SUM(CASE WHEN winner_hand = 'R' AND loser_hand = 'L' THEN 1 ELSE 0 END) /
          SUM(CASE WHEN (winner_hand = 'L' AND loser_hand = 'R') OR (winner_hand = 'R' AND loser_hand = 'L') THEN 1 ELSE 0 END), 2) AS "Right-Handed Winning %"
FROM
    matches
WHERE
    (winner_hand = 'L' AND loser_hand = 'R') OR (winner_hand = 'R' AND loser_hand = 'L');


 * sqlite:///C:/Users/chojk/Desktop/myproject/KaggleTennisDataProject/database.sqlite
Done.


metric,Left-Handed Wins,Left-Handed Winning %,Right-Handed Wins,Right-Handed Winning %
Winning Statistics,73897,51.51,69571,48.49
